In [1]:
from sage.structure.element import Element

# create a Multiplication Gate object with left, right, out, and an operation

class Gate(Element):
    '''(l) and (r) are lists of left or right input wires. Addition is compressed  (o) is
       a list containing the single output wire.'''
    def __init__(self, l, r, o, root):
        self.left = l
        self.right = r
        self.out = o
        self.root = root
        
    def __repr__(self):
        return ("Gate(" + str(self.left) + ", " + str(self.right) + ", " + self.out + ")")
    
# Parameters for BLS12-381, a pairing-friendly curve used in ZCash Sapling    
F=GF(0x1a0111ea397fe69a4b1ba7b6434bacd764774b84f38512bf6730d2a0f6b0f6241eabfffeb153ffffb9feffffffffaaab)
E=EllipticCurve(F, [0,4])

In [19]:

# circuit structure is defined with `gates` below
# this circuit is (c_1+c_2)*(c_3*c_4)=c_6
# example assignment: 2, 7, -3, 5, -135

n = 4 # inputs
s = 2 # multiplication gates
N = n + s # total number of wires

# note that this polynomial ring is over integers rather than a finite field due to limitations with sage

P.<x> = PolynomialRing(ZZ)

var('c_1 c_2 c_3 c_4 c_5 c_6')

wires = [c_1, c_2, c_3, c_4, c_5, c_6]

gates = [
    Gate([c_3], [c_4], [c_5], 0),
    Gate([c_1, c_2], [c_5], [c_6], 1)
]

roots = [g.root for g in gates]

In [28]:
# target polynomial
T = prod([x - g.root for g in gates])

# left polynomial
L = sum([c*prod([x - g.root + (1 if c in g.left else 0) for g in gates]) for c in wires])

# right polynomial
R = sum([c*prod([x - g.root + (1 if c in g.right else 0) for g in gates]) for c in wires])

# out polynomial
O = sum([c*prod([(x - g.root)^2 + (1 if c in g.out else 0) for g in gates]) for c in wires])

Q = L*R-O

print(L)

c_1*x^2 + c_2*x^2 + (x^2 - 1)*c_3 + (x^2 - x)*c_4 + (x^2 - x)*c_5 + (x^2 - x)*c_6


In [65]:
# prover knows avalid assignment: 2, 7, -3, 5, -135
# this includes intermediate step with -3 * 5 = -15 (c_5)

# program polynomial evaluated at this assignment

Qe1 = Q(2,7,-3,5,-15,-135)
print([Qe1(r) for r in roots])

# output of [0, 0] means Qe1 has roots at both roots of T, so both multiplication gates are correctly computed

[0, 0]


In [67]:
# here is an incorrect assignment

Qe2 = Q(2,7,-3,5,-15,-136)
print([Qe2(r) for r in roots])

# output of [0, 1] means Qe2 does not have a root for the second gate, so the second gate is incorrectly computed

[0, 1]


In [ ]:
# define the elliptic curve parameters for alt_bn_128
q = 21888242871839275222246405745257275088696311157297823662689037894645226208583

# define our finite fields
Fq = GF(q)
R.<t> = PolynomialRing(Fq)
Fq2.<i> = Fq.extension(t^2+1)
Fq12.<j> = Fq.extension(82-18*t^6+t^12)

# define the elliptic curves
E = EllipticCurve(Fq, [0,3])
E2 = EllipticCurve(Fq2, [0, 3/(9+i)])
E12 = EllipticCurve(Fq12, [0, 3])


# generators for our curves
G1 = E(1, 2)
G2 = E2(
  11559732032986387107991004021392285783925812861821192530917403151452391805634 * i +
  10857046999023057135944570762232829481370756359578518086990519993285655852781,
  4082367875863433681332203403145435568316851327593401208105741076214120093531 * i +
  8495653923123431417604973247489272438418190587263600148770280649306958101930
   )

In [ ]:
# new we can build up the apparatus for computing a pairing

# we are using an Ate pairing, where our two elliptic curve points 
# are on two different curves (E and E2 in this case) but are mapped
# to a third elliptic curve (E12) for the pairing arithmetic

# some precomupted numbers we need for the Ate pairing

curve_order = 21888242871839275222246405745257275088548364400416034343698204186575808495617
E12_trace_of_frobenius = 219935527718358747437685322321746888215792990022412734112037885679359160821747878808219383053039080712646717876492712517106434983527509917272700114061872443178054096459403896123212817639965365778333704429105246520228990368104811904610399806718379154632718223430673353654343444964609177304997512400034119082640719481391775619518018555403226583182026891006289337232060054232144962470325381114532246761168032255579192085445576594976954355784276425978642748189538

# this takes elliptic curve point coordinates in the form of [ai+b, ci+d]
# and returns [[a,b], [c,d]]

def coeffs(P2):
    return [[(P2[0].conjugate()+P2[0])/2, (P2[0]-(P2[0].conjugate()+P2[0])/2)/i], [(P2[1].conjugate()+P2[1])/2, (P2[1]-(P2[1].conjugate()+P2[1])/2)/i]]

# the twist maps an elliptic curve point from E2 to E12
def twist(P2):
    W = Fq12([0,1,0,0,0,0,0,0,0,0,0,0])
    xc = coeffs(P2)[0][0]-9*coeffs(P2)[0][1], coeffs(P2)[0][1]
    yc = coeffs(P2)[1][0]-9*coeffs(P2)[1][1], coeffs(P2)[1][1]
    nx = Fq12([xc[0], 0, 0, 0, 0, 0, xc[1], 0, 0, 0, 0, 0])
    ny = Fq12([yc[0], 0, 0, 0, 0, 0, yc[1], 0, 0, 0, 0, 0])
    return E12(nx * W^2, ny * W^3)

# the cast maps an elliptic curve point from E to E12
def cast(P1):
    coords = P1.xy();
    return E12(Fq12(coords[0]), Fq12(coords[1]))

# apply the maps and perform the Ate pairing in E12
def pairing(P,Q):
    return cast(P).ate_pairing(twist(Q),curve_order,12,E12_trace_of_frobenius)
